In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import OrderedDict
import pickle
import pickle5
import seaborn as sns
import folium

## block warning
import warnings
warnings.filterwarnings('ignore')

In [3]:
# under_utilization data
with open("../data/under_utilization_combinedStations_laggedV5.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,Station_Name,weekday_b,weekday_b_name,hour_b,...,near_inactivity_4H,near_charge_events_4H,near_charge_time_5H,near_charge_energy_5H,near_inactivity_5H,near_charge_events_5H,near_charge_time_6H,near_charge_energy_6H,near_inactivity_6H,near_charge_events_6H
0,928.5,2018-01-31 19:51:00,2018-02-01 11:19:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,2,Wednesday,19,...,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
1,363.5,2018-02-01 14:03:00,2018-02-01 20:06:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,3,Thursday,14,...,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
2,6828.5,2018-02-01 21:15:00,2018-02-06 15:03:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,3,Thursday,21,...,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
3,5871.5,2018-02-06 15:27:00,2018-02-10 17:18:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,1,Tuesday,15,...,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
4,1454.5,2018-02-10 18:26:00,2018-02-11 18:40:30,40.018482,-105.281066,0.152203,COMM VITALITY / 1104 SPRUCE1,5,Saturday,18,...,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0


In [4]:
## Limit to only top 10 stations
temp = df.groupby(['Station_Name']).count().latitude # group the data for each station
names4 = temp.sort_values(ascending=False)[0:4]
names4 = names4.index.values
names8 = temp.sort_values(ascending=False)[0:8]
names8 = names8.index.values

In [5]:
names = df['Station_Name'].unique()

In [6]:
coordinate_dict4 = {}
for name in names4:
    temp = df[df['Station_Name'] == name]
    coordinate_dict4[name] = [temp.latitude.unique()[0], temp.longitude.unique()[0]]
    
coordinate_dict8 = {}
for name in names8:
    temp = df[df['Station_Name'] == name]
    coordinate_dict8[name] = [temp.latitude.unique()[0], temp.longitude.unique()[0]]
    
coordinate_dict17 = {}
for name in df['Station_Name'].unique():
    temp = df[df['Station_Name'] == name]
    coordinate_dict17[name] = [temp.latitude.unique()[0], temp.longitude.unique()[0]]

### Map for all stations

In [7]:
map_B = folium.Map([40.0, -105.2705], zoom_start=14)
for name in coordinate_dict17:
    folium.Marker([coordinate_dict17[name][0] , coordinate_dict17[name][1]], popup=name).add_to(map_B)
map_B

Determine what stations are outside center and inside center.

In [8]:
outside = ['BOULDER / BASELINE ST1','BOULDER / ALPINE ST1', 'BOULDER / N BOULDER REC 1',
           'COMM VITALITY / BOULDER JCTN', 'BOULDER / JUNCTION ST1', 'BOULDER / EAST REC',
           'BOULDER / REC CENTER', 'BOULDERJUNCTION / JUNCTION ST1']
inside = list(set(names) - set(outside))

**Outside city center**

In [9]:
map_B = folium.Map([40.01, -105.2705], zoom_start=12, control_scale=True,
                   width=400, heigh=450)
# Adding points ontop of the map
[folium.CircleMarker(
                     location=[coordinate_dict17[name][0] , coordinate_dict17[name][1]], #Given all relevant coordinates 
                     radius=5, #setting the radius of the marker to 1
                     color='blue',
                     fill_color='white') #selecting the prefered color of the datapoints
                     .add_to(map_B) #adding the points to the map
                     for name in outside] # adding all the point by using a for-loop


    
nam = 'BOULDER / BASELINE ST1'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.003],
              icon=folium.DivIcon(html="<b>3<b>")
             ).add_to(map_B)

nam = 'BOULDER / ALPINE ST1'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.003],
              icon=folium.DivIcon(html="<b>1<b>")
             ).add_to(map_B)

nam =  'BOULDER / N BOULDER REC 1'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.003],
              icon=folium.DivIcon(html="<b>8<b>")
             ).add_to(map_B)

nam =  'COMM VITALITY / BOULDER JCTN'
folium.Marker(location = [coordinate_dict17[nam][0]+0.003,
              coordinate_dict17[nam][1]+0.003],
              icon=folium.DivIcon(html="<b>18<b>")
             ).add_to(map_B)

nam =  'BOULDERJUNCTION / JUNCTION ST1'
folium.Marker(location = [coordinate_dict17[nam][0]-0.002,
              coordinate_dict17[nam][1]-0.004],
              icon=folium.DivIcon(html="<b>10<b>")
             ).add_to(map_B)

nam =  'BOULDER / JUNCTION ST1'
folium.Marker(location = [coordinate_dict17[nam][0]-0.001,
              coordinate_dict17[nam][1]+0.006],
              icon=folium.DivIcon(html="<b>7<b>")
             ).add_to(map_B)

nam =  'BOULDER / EAST REC'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.003],
              icon=folium.DivIcon(html="<b>5<b>")
             ).add_to(map_B)

nam =  'BOULDER / REC CENTER'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.003],
              icon=folium.DivIcon(html="<b>9<b>")
             ).add_to(map_B)
map_B

In [10]:
#map_B.save('../Visualization/BoulderMap_outskirts.html')

**Inside city center**

In [11]:
map_B = folium.Map([40.018, -105.279157], zoom_start=15, control_scale=True,
                   width=400, heigh=450)
# Adding points ontop of the map
[folium.CircleMarker(
                     location=[coordinate_dict17[name][0] , coordinate_dict17[name][1]], #Given all relevant coordinates 
                     radius=5, #setting the radius of the marker to 1
                     color='blue',
                     fill_color='white') #selecting the prefered color of the datapoints
                     .add_to(map_B) #adding the points to the map
                     for name in inside] # adding all the point by using a for-loop


    
nam = 'COMM VITALITY / 1104 SPRUCE1'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.0009],
              icon=folium.DivIcon(html="<b>13<b>")
             ).add_to(map_B)

nam = 'BOULDER / BOULDER PARK'
folium.Marker(location = [coordinate_dict17[nam][0]-0.0004,
              coordinate_dict17[nam][1]-0.0002],
              icon=folium.DivIcon(html="<b>4<b>")
             ).add_to(map_B)

nam =  'COMM VITALITY / 1500PEARL'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]-0.0007],
              icon=folium.DivIcon(html="<b>15<b>")
             ).add_to(map_B)

nam =  'COMM VITALITY / 1100WALNUT1'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]+0.0007],
              icon=folium.DivIcon(html="<b>11<b>")
             ).add_to(map_B)

nam =  'COMM VITALITY / 2200 BROADWAY1'
folium.Marker(location = [coordinate_dict17[nam][0],
              coordinate_dict17[nam][1]+0.0007],
              icon=folium.DivIcon(html="<b>16<b>")
             ).add_to(map_B)

nam =  'BOULDER / FACILITIES ST1'
folium.Marker(location = [coordinate_dict17[nam][0]-0.0004,
              coordinate_dict17[nam][1]+0.0003],
              icon=folium.DivIcon(html="<b>6<b>")
             ).add_to(map_B)

nam =  'BOULDER / ATRIUM ST1'
folium.Marker(location = [coordinate_dict17[nam][0]+0.0005,
              coordinate_dict17[nam][1]-0.0003],
              icon=folium.DivIcon(html="<b>2<b>")
             ).add_to(map_B)

nam =  'COMM VITALITY / 1000WALNUT'
folium.Marker(location = [coordinate_dict17[nam][0]-0.0004,
              coordinate_dict17[nam][1]+0.0003],
              icon=folium.DivIcon(html="<b>12<b>")
             ).add_to(map_B)

nam =  'COMM VITALITY / 1400 WALNUT1'
folium.Marker(location = [coordinate_dict17[nam][0]-0.0004,
              coordinate_dict17[nam][1]+0.0003],
              icon=folium.DivIcon(html="<b>14<b>")
             ).add_to(map_B)
map_B

In [12]:
#map_B.save('../Visualization/BoulderMap_center.html')

### Map for the four station

In [14]:
names4_short = ['1000 WALNUT', 'N BOULDER', '1104 SPRUCE', 'BASELINE']

In [15]:
map_B = folium.Map([40.03, -105.2705], tiles="OpenStreetMap", zoom_start=12.45, control_scale=True,
                   width=400, height= 450)
# Adding points ontop of the map
[folium.CircleMarker(
                     location=[coordinate_dict4[name][0] , coordinate_dict4[name][1]], #Given all relevant coordinates 
                     radius=5, #setting the radius of the marker to 1
                     color='red',
                     fill_color='white') #selecting the prefered color of the datapoints
                     .add_to(map_B) #adding the points to the map
                     for name in coordinate_dict4] # adding all the point by using a for-loop

nam = 'COMM VITALITY / 1000WALNUT'
folium.Marker(location = [coordinate_dict4[nam][0]-0.004,
              coordinate_dict4[nam][1]-0.023],
              icon=folium.DivIcon(html="<b>1000_WALNUT<b>")
             ).add_to(map_B)

nam = 'COMM VITALITY / 1104 SPRUCE1'
folium.Marker(location = [coordinate_dict4[nam][0]+0.005,
              coordinate_dict4[nam][1]-0.012],
              icon=folium.DivIcon(html="<b>1104_SPRUCE<b>")
             ).add_to(map_B)

nam = 'BOULDER / N BOULDER REC 1'
folium.Marker(location = [coordinate_dict4[nam][0]+0.005,
              coordinate_dict4[nam][1]-0.01],
              icon=folium.DivIcon(html="<b>N_BOULDER<b>")
             ).add_to(map_B)

nam = 'BOULDER / BASELINE ST1'
folium.Marker(location = [coordinate_dict4[nam][0]-0.003,
              coordinate_dict4[nam][1]-0.008],
              icon=folium.DivIcon(html="<b>BASELINE<b>")
             ).add_to(map_B)
map_B

In [16]:
#map_B.save('../Visualization/BoulderMap4stat.html')

In [17]:
map_B = folium.Map([40.02, -105.26], tiles="OpenStreetMap", zoom_start=12.45, control_scale=True,
                   width=400, height= 450)
# Adding points ontop of the map
[folium.CircleMarker(
                     location=[coordinate_dict17[name][0] , coordinate_dict17[name][1]], #Given all relevant coordinates 
                     radius=3, #setting the radius of the marker to 1
                     color='blue',
                     fill_color='white') #selecting the prefered color of the datapoints
                     .add_to(map_B) #adding the points to the map
                     for name in coordinate_dict17] # adding all the point by using a for-loop
map_B

In [ ]:
#map_B.save('../Visualization/BoulderMap.html')